# <font color='purple'>Projeto 1 - Extraindo e Analisando Dados de Bancos de Dados</font>

### Pacotes python usados no projeto

In [1]:
# Instala o pacote watermark
!pip install -q -U watermark

In [2]:
!pip install -q ipython-sql

In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [4]:
# Imports
import os
import pandas as pd
import sqlite3

In [5]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



### Criando o Banco de Dados Relacional


In [6]:
# Cria um dataframe com os dados de origem
dados = pd.DataFrame({'nivel': ['Junior', 'Pleno', 'Senior'],
                      'salario': [7500, 14650, 18320],
                      'cargo': ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']})

In [7]:
# Lista os dados
dados.head()

,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [10]:
# Define o caminho para o arquivo do banco de dados
arquivo_path = 'dsadatabase.db'

In [11]:
# Verifica se o arquivo existe e deleta se existir para criar um novo arquivo depois
if os.path.exists(arquivo_path):
    try:
        os.remove(arquivo_path)
        print(f"Arquivo {arquivo_path} deletado com sucesso!")
    except Exception as e:
        print(f"Erro ao deletar o arquivo {arquivo_path}. Detalhes: {e}")
else:
    print(f"Arquivo {arquivo_path} não encontrado.")

Arquivo dsadatabase.db não encontrado.


### Conectando ao Banco de Dados com Linguagem Python


In [12]:
# Criamos a conexão a um banco de dados SQLite
cnn = sqlite3.connect('dsadatabase.db')

In [13]:
# Copia o dataframe para dentro do banco de dados como uma tabela
dados.to_sql('funcionarios', cnn)

3

In [14]:
# Carregamos a extensão SQL
%load_ext sql

In [15]:
# Definimos o banco de dados
%sql sqlite:///dsadatabase.db

> Agora executamos nossas consultas SQL usando diretamente Linguagem SQL dentro do Jupyter Notebook.


In [16]:
%%sql

SELECT * FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


index,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [17]:
%%sql
SELECT count(*) FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


count(*)
3


In [18]:
%%sql
SELECT round(avg(salario),2) as 'salario_medio' FROM funcionarios

 * sqlite:///dsadatabase.db
Done.


salario_medio
13490.0


### Carregando o Banco de Dados a Partir de Arquivos CSV
<p> Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Vamos colocar o conteúdo do arquivo em banco de dados.

In [24]:
# Carrega o dataset
df = pd.read_csv('dataset.csv')

In [25]:
type(df)

pandas.core.frame.DataFrame

In [26]:
df.shape

(768, 9)

In [27]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,103,30,38,83,43.3,0.183,33,0
1,1,115,70,30,96,34.6,0.529,32,1
2,3,126,88,41,235,39.3,0.704,27,0
3,8,99,84,0,0,35.4,0.388,50,0
4,7,196,90,0,0,39.8,0.451,41,1


In [28]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

768

In [29]:
%%sql

SELECT count(*) FROM diabetes

 * sqlite:///dsadatabase.db
Done.


count(*)
768


### Sintaxe SQL e Sintaxe Pandas

> Retorne os pacientes que têm BMI maior que 52 e idade (age) entre 25 a 30 anos

### Sintaxe de consulta do Pandas:

In [30]:
df.query("BMI > 52 and 25 <= Age <= 30")

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.420,25,1


### Sintaxe de consulta com SQL

In [32]:
%%sql

SELECT * from diabetes WHERE BMI > 52 AND age BETWEEN 25 AND 30

 * sqlite:///dsadatabase.db
Done.


index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.42,25,1
